In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from reconstruction import Model, TopKLayer
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import synthesizer
import os
import itertools
from torch import optim

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [10]:
def getImage(img_path, p, epochs=100):
    m = Model(img_path, topk=p['topk'], device=p['device'], dimensions=p['dimensions'], mode=p['mode'], forward=False)
    I = torch.ones((1, 3, m.dimensions[0], m.dimensions[1])).to(device)
    I = I.requires_grad_(True)
    optimizer = optim.LBFGS([I], lr=1)

    r = 0
    while r < epochs:
        def closure():
            optimizer.zero_grad()
            I.grad = None
            m(I)
            loss = m.loss()
            loss.backward(retain_graph=True)
            #print(f'I gradient: {I.grad}')
            return loss
        optimizer.step(closure)
        
        #transforms.ToPILImage()(np.clip(I.clone().detach().cpu().squeeze(0).numpy().transpose(1, 2, 0), 0, 1)).save(os.path.join('./results', f'{r}.jpg'))
        r += 1
        print(f'\rEpoch {r}: Loss {m.loss().item()}', end='')
    print()
    return I

In [11]:
def get_path(img_name):
    img_ext = '.jpg'
    img_path = os.path.join('./', img_name + img_ext)
    return img_path, img_name, img_ext

In [12]:
parameters = {
    'topk': 0.05,
    'device': device,
    'dimensions': (500, 500), 
    'mode': 'topk'
} 

In [13]:
topks = [0.05]#, 0.5, 0.95]
modes = ['non-topk', 'topk', 'both']
img_names = ['rocks']#, 'jeep1']

I = None
for p in itertools.product(topks, modes, img_names):
    print(p)
    topk = p[0]
    mode = p[1]
    img_path, img_name, img_ext = get_path(p[2])
    
    parameters['topk'] = topk
    parameters['mode'] = mode

    I = getImage(img_path, parameters, epochs=10)
    transforms.ToPILImage()(np.clip(I.clone().detach().cpu().squeeze(0).numpy().transpose(1, 2, 0), 0, 1)).save(os.path.join('./results', f'{img_name}-{topk}-{mode}{img_ext}'))

(0.05, 'non-topk', 'rocks')


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [8]:
I

tensor([[[[ 0.9632,  0.9556,  0.9069,  ...,  1.0904,  1.0969,  1.0275],
          [ 0.9479,  0.9264,  0.9621,  ...,  1.0080,  1.0986,  1.0398],
          [ 0.9314,  0.8927,  0.8487,  ...,  1.0670,  1.0733,  1.0419],
          ...,
          [ 0.0481,  0.1147,  0.0648,  ...,  0.0143,  0.2033,  0.1898],
          [ 0.2544,  0.2448,  0.2617,  ...,  0.1273,  0.1984,  0.2398],
          [ 0.1596,  0.1689,  0.3134,  ...,  0.0694,  0.1932,  0.1275]],

         [[ 0.9887,  0.9251,  0.9337,  ...,  1.0523,  1.0238,  1.0962],
          [ 0.9023,  0.7913,  0.9152,  ...,  1.1042,  1.0354,  1.1130],
          [ 0.7753,  0.7621,  0.7689,  ...,  1.0715,  1.0517,  1.0718],
          ...,
          [ 0.5221,  0.2315,  0.2933,  ...,  0.2332,  0.2353,  0.2695],
          [ 0.2344,  0.0957,  0.4308,  ...,  0.1088,  0.1323,  0.3252],
          [ 0.3659,  0.5201,  0.5370,  ...,  0.1574,  0.2540,  0.2327]],

         [[ 0.4994,  0.4360,  0.3460,  ...,  1.0445,  1.1304,  1.0426],
          [ 0.5023,  0.4637,  